In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import h5py
import cv2
import os
import glob
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adadelta, Adam
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, History
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import PIL
from __future__ import division, print_function
from collections import Counter
from scipy.misc import imread
from scipy.misc import imshow
import random
%matplotlib inline

In [ ]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

def VGG_16(size=(224, 224), weights_path='data/vgg16_bn_conv.txt'):
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,)+size))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))


    if weights_path:
        model.load_weights(weights_path)

    return model

In [ ]:
size = (360, 270)
model = VGG_16(size=size)


for layer in model.layers:
    layer.trainable = False

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(16, activation='softmax'))

In [ ]:
model.save_weights('vgg16_group_order_v2.pkl')

In [ ]:
bath_size = 8
random_seed = np.random.random_integers(0, 100000)
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
            'data/test2/',
            target_size=size,
            batch_size=bath_size,
            shuffle = False,
            seed = random_seed,
            classes = None,
            class_mode = None)

In [ ]:
nb_test_samples = len(glob.glob('data/test2/*/*.jpg'))+len(glob.glob('data/test2/*/*/*.jpg'))
prediction = model.predict_generator(test_generator, nb_test_samples)
test_image_list = test_generator.filenames

In [ ]:
train_ship_dict={}
for i in range(0,len(test_image_list)):
    temp_index=0
    for j in range(0,prediction.shape[1]):
        if prediction[i][j]==max(prediction[i]):
            temp_index=j+1
    train_ship_dict[test_image_list[i]]=temp_index

In [ ]:
for train_pic in train_ship_dict:
   os.rename('data/test2 _tem/'+train_pic, 'data/test2 _tem/' +str(train_ship_dict[train_pic])+'/'+train_pic.split('/')[1])

In [ ]:
crop_dict={
1:
(0.082, 0.347, 0.586, 1),
2:
(0.375, 0, 0.555, 0.306),
3:
(0.328, 0.642, 0.563, 1),
4:
(0.078, 0, 0.781, 0.875),
5:
(0.156, 0, 0.75, 0.867),
6:
(0.375, 0.319, 0.797, 1),
7:
((0,0,0.563,1),    (0.437,0,1,1,)),
8:
(0.356, 0, 0.784, 0.703),
9:
(0, 0, 0.503, 0.895),
10:
(0.437,0,1,1),
11:
(0,0,0.563,1),
12:
(0.437,0,1,1),
13:
(0,0,1,1),
14:
(0.436, 0, 0.864, 0.657),
15:
(0,0,1,1),
16:
(0.336, 0.153, 0.766, 0.861)
}

In [ ]:
for train_pic in train_ship_dict:
    if train_ship_dict[train_pic] != 7:
        img = PIL.Image.open('data/test/'+train_pic)  
        croped = img.crop((int(img.width*crop_dict[train_ship_dict[train_pic]][0]),int(img.height*crop_dict[train_ship_dict[train_pic]][1]),int(img.width*crop_dict[train_ship_dict[train_pic]][2]),int(img.height*crop_dict[train_ship_dict[train_pic]][3])))
        croped.save('data/test_crop/'+train_pic)

In [ ]:
for train_pic in train_ship_dict:
    if train_ship_dict[train_pic] == 7:
        img = PIL.Image.open('data/test/'+train_pic)  
        croped = img.crop((int(img.width*crop_dict[train_ship_dict[train_pic]][0][0]),int(img.height*crop_dict[train_ship_dict[train_pic]][0][1]),int(img.width*crop_dict[train_ship_dict[train_pic]][0][2]),int(img.height*crop_dict[train_ship_dict[train_pic]][0][3])))
        croped.save('data/test_ship7_train/'+train_pic)

In [ ]:
for train_pic in train_ship_dict:
    if train_ship_dict[train_pic] == 7:
        img = PIL.Image.open('data/test/'+train_pic)  
        croped = img.crop((int(img.width*crop_dict[train_ship_dict[train_pic]][1][0]),int(img.height*crop_dict[train_ship_dict[train_pic]][1][1]),int(img.width*crop_dict[train_ship_dict[train_pic]][1][2]),int(img.height*crop_dict[train_ship_dict[train_pic]][1][3])))
        croped.save('data/test_ship7_train_2/'+train_pic)

In [ ]:
img_glob=glob.glob('data/ship7_train_2/*/*')
for img_path in img_glob:
    img = PIL.Image.open(img_path)
    img.save('data/final_train/'+img_path.split('/')[2]+'/'+str(random.randint(0,9999999999999999))+'.jpg')

In [ ]:
# img_glob=glob.glob('data/internetFish_back/*/*')
for img_path in img_glob:
    os.rename(img_path, 'data/final_train/'+img_path.split('/')[2]+'/'+str(random.randint(0,9999999999999999))+'.jpg')